In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import max_error
from utils.utils import generate_synthetic_data, generate_final_features, split_dataset
import warnings 
warnings.filterwarnings("ignore")


: 

In [ ]:
# load the data
total_regular_cycle_data = pd.read_csv('dataset\\total_regular_cycle_data.csv')
#regular_cycle_data = pd.read_csv('dataset\\regular_cycle_data.csv')
#irregular_cycle_data = pd.read_csv('dataset\irregular_cycle_data.csv')

: 

In [ ]:
total_regular_cycle_data.head()

: 

In [ ]:
# generate the final features
features_total_regular_cycle_data, labels_total_regular_cycle_data = generate_final_features(total_regular_cycle_data)
features_regular_cycle_data, labels_regular_cycle_data = generate_final_features(regular_cycle_data)
features_irregular_cycle_data, labels_irregular_cycle_data = generate_final_features(irregular_cycle_data)


: 

In [ ]:
# slice the data with split_dataset function
input_train_total_regular_cycle, input_test_total_regular_cycle, output_train_total_regular_cycle, output_test_total_regular_cycle = split_dataset(features_total_regular_cycle_data, labels_total_regular_cycle_data, reshape=False)
#input_train_regular_cycle, input_test_regular_cycle, output_train_regular_cycle, output_test_regular_cycle = split_dataset(features_regular_cycle_data, labels_regular_cycle_data)
#input_train_irregular_cycle, input_test_irregular_cycle, output_train_irregular_cycle, output_test_irregular_cycle = split_dataset(features_irregular_cycle_data, labels_irregular_cycle_data)


: 

In [ ]:
input_train_total_regular_cycle.shape

: 

In [ ]:
# create dataset
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(input_train_total_regular_cycle, look_back)
testX, testY = create_dataset(input_test_total_regular_cycle, look_back)

: 

In [ ]:
plt.plot(output_train_total_regular_cycle[:,0])
plt.plot(input_train_total_regular_cycle[:,0])

: 

: 

In [ ]:
# create and fit the LSTM network
n_features = input_train_total_regular_cycle.shape[2]
model = Sequential()
model.add(LSTM(4, input_shape=(3, n_features),  activation='tanh'))
model.add(Dense(n_features, activation='relu'))

opt=tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mae', 'mape'])

model.fit(input_train_total_regular_cycle, output_train_total_regular_cycle, epochs=100)

: 

In [ ]:
# make predictions
trainPredict = model.predict(input_train_total_regular_cycle)
testPredict = model.predict(input_test_total_regular_cycle)

: 

In [ ]:
# calculate root mean squared error
trainScore = np.sqrt(mean_squared_error(output_train_total_regular_cycle , trainPredict))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(output_test_total_regular_cycle, testPredict))
print('Test Score: %.2f RMSE' % (testScore))


: 

In [ ]:
# plot baseline and predictions
plt.plot(testPredict)



: 

In [ ]:
# implementation of a LSTM model with tensorflow for time series prediction

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import max_error



# load data
df = pd.read_csv('data.csv', index_col=0)
df.index = pd.to_datetime(df.index)
df = df.resample('D').mean()
df = df.interpolate(method='linear')
df = df.fillna(0)
df = df.dropna()

# split data
train_size = int(len(df) * 0.8)
test_size = len(df) - train_size
train, test = df.iloc[0:train_size], df.iloc[train_size:len(df)]

# scale data
scaler = MinMaxScaler(feature_range=(0, 1))
train = scaler.fit_transform(train)
test = scaler.transform(test)

# create dataset
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])


# calculate root mean squared error
trainScore = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

# calculate mean absolute error
trainScore = mean_absolute_error(trainY[0], trainPredict[:,0])
print('Train Score: %.2f MAE' % (trainScore))
testScore = mean_absolute_error(testY[0], testPredict[:,0])
print('Test Score: %.2f MAE' % (testScore))

# calculate r2 score
trainScore = r2_score(trainY[0], trainPredict[:,0])
print('Train Score: %.2f R2' % (trainScore))
testScore = r2_score(testY[0], testPredict[:,0])
print('Test Score: %.2f R2' % (testScore))

# calculate explained variance score
trainScore = explained_variance_score(trainY[0], trainPredict[:,0])
print('Train Score: %.2f EVS' % (trainScore))
testScore = explained_variance_score(testY[0], testPredict[:,0])
print('Test Score: %.2f EVS' % (testScore))

# calculate max error
trainScore = max_error(trainY[0], trainPredict[:,0])
print('Train Score: %.2f Max Error' % (trainScore))
testScore = max_error(testY[0], testPredict[:,0])
print('Test Score: %.2f Max Error' % (testScore))


# shift train predictions for plotting
trainPredictPlot = np.empty_like(df)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

# shift test predictions for plotting
testPredictPlot = np.empty_like(df)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(df)-1, :] = testPredict

# plot baseline and predictions
plt.plot(scaler.inverse_transform(df))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

# save model
model.save('model.h5')

# save scaler
import pickle
pickle.dump(scaler, open('scaler.pkl', 'wb'))

# save predictions
df = pd.DataFrame(testPredict)
df.to_csv('predictions.csv')

# save test data
df = pd.DataFrame(testY[0])
df.to_csv('test_data.csv')

# save train data
df = pd.DataFrame(trainY[0])
df.to_csv('train_data.csv')




: 